**Imports**

In [1]:
#!pip install pydrive
#!pip install gdown pandas

In [2]:
#Importing libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import io
import requests
import json
warnings.filterwarnings('ignore')


from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import pandas as pd
import io
import gdown
import zipfile
import os


**Authenticate google drive**


In [3]:
# gauth = GoogleAuth()
# gauth.LocalWebserverAuth()  # or gauth.CommandLineAuth() depending on environment
# drive = GoogleDrive(gauth)


In [4]:
# Replace with your actual sheet-ID and sheet GID
sheet_id = "1nBiruECDXrBd7BgX4gStJXWKFYBNU_IErU5nP7sRih0"
gid = "133044706"

url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}"
df = pd.read_csv(url)

df.head()


,boxes_box1_gps,boxes_crop,boxes_planting_date,boxes_expected_harvest_date,country,boxes_farmer_district,wet_completed_time,wet_kgs_crop_box1,wet_kgs_crop_box2,dry_box1_dry_weight,dry_box2_dry_weight,Latitude,Longitude,Altitude,Accuracy
0,-0.499127 37.6122529 1252.08 7.62,maize,2021-10-06,2022-03-25,Kenya,Embu,2022-03-20,4.910,8.810,2.600,3.395,-0.499127,37.612253,1252.08,7.62
1,-0.4754511 37.6402894 1234.5 9.73,maize,2021-11-10,2022-03-12,Kenya,Embu,2022-03-03,12.170,10.520,3.195,4.415,-0.475451,37.640289,1234.50,9.73
2,-0.5351206 37.6086503 1226.4 4.5,maize,2021-12-02,2022-02-26,Kenya,Embu,2022-02-23,1.245,1.575,0.495,0.535,-0.535121,37.608650,1226.40,4.50
3,-0.3971582 37.5027862 1839.7 5.25,maize,2021-11-06,2022-04-06,Kenya,Embu,2022-04-06,3.700,3.925,3.555,3.470,-0.397158,37.502786,1839.70,5.25
4,-0.4509697 37.6600589 1234.53 8.0,maize,2021-11-14,2022-04-01,Kenya,Embu,2022-03-25,4.645,4.815,3.605,3.695,-0.450970,37.660059,1234.53,8.00


In [5]:
# Preview

print("\nColumns:", df.columns.tolist())
print("\nShape:", df.shape)


Columns: ['boxes_box1_gps', 'boxes_crop', 'boxes_planting_date', 'boxes_expected_harvest_date', 'country', 'boxes_farmer_district', 'wet_completed_time', 'wet_kgs_crop_box1', 'wet_kgs_crop_box2', 'dry_box1_dry_weight', 'dry_box2_dry_weight', 'Latitude', 'Longitude', 'Altitude', 'Accuracy']

Shape: (428, 15)


In [6]:
# Add a unique farm_id starting from 1
df.insert(0, 'Plot_No', range(1, len(df) + 1))

**Clean and Rename columns**

In [7]:
# Drop the unwanted column
df = df.drop(columns=['boxes_box1_gps'], errors='ignore')

# Rename specific columns
df = df.rename(columns={
    'boxes_crop': 'crop_type',
    'boxes_planting_date': 'planting_date',
    'boxes_expected_harvest_date': 'expected_harvest_date',
    'boxes_farmer_district': 'county'
})


# Convert planting and harvest dates to datetime
df['planting_date'] = pd.to_datetime(df['planting_date'], errors='coerce')
df['expected_harvest_date'] = pd.to_datetime(df['expected_harvest_date'], errors='coerce')
df['wet_completed_time'] = pd.to_datetime(df['wet_completed_time'], errors='coerce')

In [8]:
df.head()

,Plot_No,crop_type,planting_date,expected_harvest_date,country,county,wet_completed_time,wet_kgs_crop_box1,wet_kgs_crop_box2,dry_box1_dry_weight,dry_box2_dry_weight,Latitude,Longitude,Altitude,Accuracy
0,1,maize,2021-10-06,2022-03-25,Kenya,Embu,2022-03-20,4.910,8.810,2.600,3.395,-0.499127,37.612253,1252.08,7.62
1,2,maize,2021-11-10,2022-03-12,Kenya,Embu,2022-03-03,12.170,10.520,3.195,4.415,-0.475451,37.640289,1234.50,9.73
2,3,maize,2021-12-02,2022-02-26,Kenya,Embu,2022-02-23,1.245,1.575,0.495,0.535,-0.535121,37.608650,1226.40,4.50
3,4,maize,2021-11-06,2022-04-06,Kenya,Embu,2022-04-06,3.700,3.925,3.555,3.470,-0.397158,37.502786,1839.70,5.25
4,5,maize,2021-11-14,2022-04-01,Kenya,Embu,2022-03-25,4.645,4.815,3.605,3.695,-0.450970,37.660059,1234.53,8.00


**Order columns**

In [9]:
# Define the preferred column order
preferred_order = ['Plot_No', 'Longitude','Latitude', 'Altitude','crop_type', 'planting_date','wet_completed_time', 'expected_harvest_date']

# Add remaining columns to the order dynamically
remaining_cols = [col for col in df.columns if col not in preferred_order]
df = df[preferred_order + remaining_cols]

In [10]:
df.head()

,Plot_No,Longitude,Latitude,Altitude,crop_type,planting_date,wet_completed_time,expected_harvest_date,country,county,wet_kgs_crop_box1,wet_kgs_crop_box2,dry_box1_dry_weight,dry_box2_dry_weight,Accuracy
0,1,37.612253,-0.499127,1252.08,maize,2021-10-06,2022-03-20,2022-03-25,Kenya,Embu,4.910,8.810,2.600,3.395,7.62
1,2,37.640289,-0.475451,1234.50,maize,2021-11-10,2022-03-03,2022-03-12,Kenya,Embu,12.170,10.520,3.195,4.415,9.73
2,3,37.608650,-0.535121,1226.40,maize,2021-12-02,2022-02-23,2022-02-26,Kenya,Embu,1.245,1.575,0.495,0.535,4.50
3,4,37.502786,-0.397158,1839.70,maize,2021-11-06,2022-04-06,2022-04-06,Kenya,Embu,3.700,3.925,3.555,3.470,5.25
4,5,37.660059,-0.450970,1234.53,maize,2021-11-14,2022-03-25,2022-04-01,Kenya,Embu,4.645,4.815,3.605,3.695,8.00


**Compute average wet and dry harvests**

In [11]:
# Ensure numeric conversion to avoid string issues
df['wet_kgs_crop_box1'] = pd.to_numeric(df['wet_kgs_crop_box1'], errors='coerce')
df['wet_kgs_crop_box2'] = pd.to_numeric(df['wet_kgs_crop_box2'], errors='coerce')
df['dry_box1_dry_weight'] = pd.to_numeric(df['dry_box1_dry_weight'], errors='coerce')
df['dry_box2_dry_weight'] = pd.to_numeric(df['dry_box2_dry_weight'], errors='coerce')

In [12]:
# Compute averages
# Each box = 40 m² → total of 80 m² for two boxes
box_area = 40
total_area = 2 * box_area  # 80 m²
m2_per_hectare = 10000

# Ensure numeric conversion to avoid issues
for col in ['wet_kgs_crop_box1', 'wet_kgs_crop_box2', 'dry_box1_dry_weight', 'dry_box2_dry_weight']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Compute total yield and convert to per hectare
df['wet_harvest'] = ((df['wet_kgs_crop_box1'] + df['wet_kgs_crop_box2']) / total_area) * m2_per_hectare
df['dry_harvest'] = ((df['dry_box1_dry_weight'] + df['dry_box2_dry_weight']) / total_area) * m2_per_hectare

# Round for clarity
df['wet_harvest_kg/ha'] = df['wet_harvest'].round(2)
df['dry_harvest_kg/ha'] = df['dry_harvest'].round(2)


In [13]:
df.head()

,Plot_No,Longitude,Latitude,Altitude,crop_type,planting_date,wet_completed_time,expected_harvest_date,country,county,wet_kgs_crop_box1,wet_kgs_crop_box2,dry_box1_dry_weight,dry_box2_dry_weight,Accuracy,wet_harvest,dry_harvest,wet_harvest_kg/ha,dry_harvest_kg/ha
0,1,37.612253,-0.499127,1252.08,maize,2021-10-06,2022-03-20,2022-03-25,Kenya,Embu,4.910,8.810,2.600,3.395,7.62,1715.000,749.375,1715.00,749.38
1,2,37.640289,-0.475451,1234.50,maize,2021-11-10,2022-03-03,2022-03-12,Kenya,Embu,12.170,10.520,3.195,4.415,9.73,2836.250,951.250,2836.25,951.25
2,3,37.608650,-0.535121,1226.40,maize,2021-12-02,2022-02-23,2022-02-26,Kenya,Embu,1.245,1.575,0.495,0.535,4.50,352.500,128.750,352.50,128.75
3,4,37.502786,-0.397158,1839.70,maize,2021-11-06,2022-04-06,2022-04-06,Kenya,Embu,3.700,3.925,3.555,3.470,5.25,953.125,878.125,953.12,878.12
4,5,37.660059,-0.450970,1234.53,maize,2021-11-14,2022-03-25,2022-04-01,Kenya,Embu,4.645,4.815,3.605,3.695,8.00,1182.500,912.500,1182.50,912.50


**Drop individual box columns now that averages are computed**

In [14]:

df = df.drop(columns=[
    'wet_kgs_crop_box1',
    'wet_kgs_crop_box2',
    'dry_box1_dry_weight',
    'dry_box2_dry_weight'
], errors='ignore')


In [15]:
df.head()

,Plot_No,Longitude,Latitude,Altitude,crop_type,planting_date,wet_completed_time,expected_harvest_date,country,county,Accuracy,wet_harvest,dry_harvest,wet_harvest_kg/ha,dry_harvest_kg/ha
0,1,37.612253,-0.499127,1252.08,maize,2021-10-06,2022-03-20,2022-03-25,Kenya,Embu,7.62,1715.000,749.375,1715.00,749.38
1,2,37.640289,-0.475451,1234.50,maize,2021-11-10,2022-03-03,2022-03-12,Kenya,Embu,9.73,2836.250,951.250,2836.25,951.25
2,3,37.608650,-0.535121,1226.40,maize,2021-12-02,2022-02-23,2022-02-26,Kenya,Embu,4.50,352.500,128.750,352.50,128.75
3,4,37.502786,-0.397158,1839.70,maize,2021-11-06,2022-04-06,2022-04-06,Kenya,Embu,5.25,953.125,878.125,953.12,878.12
4,5,37.660059,-0.450970,1234.53,maize,2021-11-14,2022-03-25,2022-04-01,Kenya,Embu,8.00,1182.500,912.500,1182.50,912.50


**Extracting Planting Season**

In [16]:
# Convert planting_date to datetime
df["planting_date"] = pd.to_datetime(df["planting_date"])

In [17]:
# Define function to determine season
def get_season(date):
    month = date.month
    if month in [3, 4, 5]:
        return "Long Rains"
    elif month in [10, 11, 12]:
        return "Short Rains"
    else:
        return "Off Season"

In [18]:
# Apply function to create 'season' column
df["season"] = df["planting_date"].apply(get_season)
df.head()

,Plot_No,Longitude,Latitude,Altitude,crop_type,planting_date,wet_completed_time,expected_harvest_date,country,county,Accuracy,wet_harvest,dry_harvest,wet_harvest_kg/ha,dry_harvest_kg/ha,season
0,1,37.612253,-0.499127,1252.08,maize,2021-10-06,2022-03-20,2022-03-25,Kenya,Embu,7.62,1715.000,749.375,1715.00,749.38,Short Rains
1,2,37.640289,-0.475451,1234.50,maize,2021-11-10,2022-03-03,2022-03-12,Kenya,Embu,9.73,2836.250,951.250,2836.25,951.25,Short Rains
2,3,37.608650,-0.535121,1226.40,maize,2021-12-02,2022-02-23,2022-02-26,Kenya,Embu,4.50,352.500,128.750,352.50,128.75,Short Rains
3,4,37.502786,-0.397158,1839.70,maize,2021-11-06,2022-04-06,2022-04-06,Kenya,Embu,5.25,953.125,878.125,953.12,878.12,Short Rains
4,5,37.660059,-0.450970,1234.53,maize,2021-11-14,2022-03-25,2022-04-01,Kenya,Embu,8.00,1182.500,912.500,1182.50,912.50,Short Rains


In [19]:
# Reorder columns — place 'season' right after 'planting_date'
cols = df.columns.tolist()
insert_pos = cols.index("planting_date") + 1
cols.insert(insert_pos, cols.pop(cols.index("season")))
df = df[cols]

# Display the DataFrame
df.head()

,Plot_No,Longitude,Latitude,Altitude,crop_type,planting_date,season,wet_completed_time,expected_harvest_date,country,county,Accuracy,wet_harvest,dry_harvest,wet_harvest_kg/ha,dry_harvest_kg/ha
0,1,37.612253,-0.499127,1252.08,maize,2021-10-06,Short Rains,2022-03-20,2022-03-25,Kenya,Embu,7.62,1715.000,749.375,1715.00,749.38
1,2,37.640289,-0.475451,1234.50,maize,2021-11-10,Short Rains,2022-03-03,2022-03-12,Kenya,Embu,9.73,2836.250,951.250,2836.25,951.25
2,3,37.608650,-0.535121,1226.40,maize,2021-12-02,Short Rains,2022-02-23,2022-02-26,Kenya,Embu,4.50,352.500,128.750,352.50,128.75
3,4,37.502786,-0.397158,1839.70,maize,2021-11-06,Short Rains,2022-04-06,2022-04-06,Kenya,Embu,5.25,953.125,878.125,953.12,878.12
4,5,37.660059,-0.450970,1234.53,maize,2021-11-14,Short Rains,2022-03-25,2022-04-01,Kenya,Embu,8.00,1182.500,912.500,1182.50,912.50


**SAve a few columns to be used in Visual crossing for planting date**

In [20]:
# Assuming df already exists and has 'season' column
selected_columns = [
    "Plot_No",
    "Longitude",
    "Latitude",
    "Altitude",
    "crop_type",
    "planting_date",
    "season",
    "country",
    "county"
]

# Select only the specified columns
df_selected = df[selected_columns]

# Save to CSV
output_path = "selected_farm_data.csv"
df_selected.to_csv(output_path, index=False)

In [21]:
df_selected.head()

,Plot_No,Longitude,Latitude,Altitude,crop_type,planting_date,season,country,county
0,1,37.612253,-0.499127,1252.08,maize,2021-10-06,Short Rains,Kenya,Embu
1,2,37.640289,-0.475451,1234.50,maize,2021-11-10,Short Rains,Kenya,Embu
2,3,37.608650,-0.535121,1226.40,maize,2021-12-02,Short Rains,Kenya,Embu
3,4,37.502786,-0.397158,1839.70,maize,2021-11-06,Short Rains,Kenya,Embu
4,5,37.660059,-0.450970,1234.53,maize,2021-11-14,Short Rains,Kenya,Embu


## **Weather Data Extraction**

In [22]:
import time
import pandas as pd
import requests
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

**Visual Crossing API KEY**

In [23]:
API_KEY = "HV8ZYXKJHRFAHLLBRY6EP87PQ"
SHEET_URL = "https://docs.google.com/spreadsheets/d/1nBiruECDXrBd7BgX4gStJXWKFYBNU_IErU5nP7sRih0/export?format=csv&gid=133044706"

In [24]:
# Retry settings
MAX_RETRIES = 5
BACKOFF_FACTOR = 5  

In [25]:
df.columns = df.columns.str.strip().str.lower()
df = df.dropna(subset=['latitude', 'longitude', 'planting_date', 'wet_completed_time'])
df['planting_date'] = pd.to_datetime(df['planting_date'], errors='coerce')
df['wet_completed_time'] = pd.to_datetime(df['wet_completed_time'], errors='coerce')
print(f"✅ Loaded {len(df)} farms")

✅ Loaded 428 farms


In [26]:
df.head()

,plot_no,longitude,latitude,altitude,crop_type,planting_date,season,wet_completed_time,expected_harvest_date,country,county,accuracy,wet_harvest,dry_harvest,wet_harvest_kg/ha,dry_harvest_kg/ha
0,1,37.612253,-0.499127,1252.08,maize,2021-10-06,Short Rains,2022-03-20,2022-03-25,Kenya,Embu,7.62,1715.000,749.375,1715.00,749.38
1,2,37.640289,-0.475451,1234.50,maize,2021-11-10,Short Rains,2022-03-03,2022-03-12,Kenya,Embu,9.73,2836.250,951.250,2836.25,951.25
2,3,37.608650,-0.535121,1226.40,maize,2021-12-02,Short Rains,2022-02-23,2022-02-26,Kenya,Embu,4.50,352.500,128.750,352.50,128.75
3,4,37.502786,-0.397158,1839.70,maize,2021-11-06,Short Rains,2022-04-06,2022-04-06,Kenya,Embu,5.25,953.125,878.125,953.12,878.12
4,5,37.660059,-0.450970,1234.53,maize,2021-11-14,Short Rains,2022-03-25,2022-04-01,Kenya,Embu,8.00,1182.500,912.500,1182.50,912.50


In [27]:

def fetch_weather(row):
    plot_no = row['farm_id'] if 'farm_id' in row else row['plot_no']
    lat, lon = row['latitude'], row['longitude']
    start_date = pd.to_datetime(row['planting_date']).strftime('%Y-%m-%d')
    end_date = pd.to_datetime(row['wet_completed_time']).strftime('%Y-%m-%d')

    url = (
        f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/"
        f"timeline/{lat},{lon}/{start_date}/{end_date}"
        f"?unitGroup=metric&include=days&key={API_KEY}&contentType=json"
    )

    retries = 0
    while retries < MAX_RETRIES:
        try:
            resp = requests.get(url, timeout=30)
            if resp.status_code == 200:
                data = resp.json()

                return [
                    {
                        "Plot_No": plot_no,
                        "Latitude": lat,
                        "Longitude": lon,
                        "Date": pd.to_datetime(day["datetime"]),
                        "Max_Temp_C": day.get("tempmax"),
                        "Min_Temp_C": day.get("tempmin"),
                        "Mean_Temp_C": day.get("temp"),
                        "Precip_mm": day.get("precip", 0),
                        "Humidity_%": day.get("humidity"),
                        "SolarEnergy_MJm2": day.get("solarenergy"),
                        "Evapotranspiration_mm": day.get("et"),
                        "WindSpeed_kmh": day.get("windspeed"),
                        "CloudCover_%": day.get("cloudcover"),
                    }
                    for day in data.get("days", [])
                ]

            elif resp.status_code == 429:
                wait_time = BACKOFF_FACTOR * (2 ** retries)
                print(f"⚠️ {plot_no} — Rate limit hit (429). Retrying in {wait_time}s...")
                time.sleep(wait_time)
                retries += 1

            else:
                print(f"❌ {plot_no} — Error {resp.status_code}: {resp.text}")
                return []

        except requests.exceptions.RequestException as e:
            wait_time = BACKOFF_FACTOR * (2 ** retries)
            print(f"⚠️ {plot_no} — Network error: {e}. Retrying in {wait_time}s...")
            time.sleep(wait_time)
            retries += 1

    print(f"❌ {plot_no} — Max retries reached. Skipping.")
    return []


**EXECUTION BLOCK**

In [28]:
def fetch_weather(row):
    plot_no = row['farm_id'] if 'farm_id' in row else row['plot_no']
    lat, lon = row['latitude'], row['longitude']
    planting_date = pd.to_datetime(row['planting_date'])
    start_date = planting_date.strftime('%Y-%m-%d')
    end_date = pd.to_datetime(row['wet_completed_time']).strftime('%Y-%m-%d')

    url = (
        f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/"
        f"timeline/{lat},{lon}/{start_date}/{end_date}"
        f"?unitGroup=metric&include=days&key={API_KEY}&contentType=json"
    )

    retries = 0
    while retries < MAX_RETRIES:
        try:
            resp = requests.get(url, timeout=30)
            if resp.status_code == 200:
                data = resp.json()

                return [
                    {
                        "Plot_No": plot_no,
                        "Latitude": lat,
                        "Longitude": lon,
                        "Planting_Date": planting_date.strftime('%Y-%m-%d'),
                        "Date": pd.to_datetime(day["datetime"]),
                        "Max_Temp_C": day.get("tempmax"),
                        "Min_Temp_C": day.get("tempmin"),
                        "Mean_Temp_C": day.get("temp"),
                        "Precip_mm": day.get("precip", 0),
                        "Humidity_%": day.get("humidity"),
                        "SolarEnergy_MJm2": day.get("solarenergy"),
                        "Evapotranspiration_mm": day.get("et"),
                        "WindSpeed_kmh": day.get("windspeed"),
                        "CloudCover_%": day.get("cloudcover"),
                    }
                    for day in data.get("days", [])
                ]

            elif resp.status_code == 429:
                wait_time = BACKOFF_FACTOR * (2 ** retries)
                print(f"⚠️ {plot_no} — Rate limit hit (429). Retrying in {wait_time}s...")
                time.sleep(wait_time)
                retries += 1

            else:
                print(f"❌ {plot_no} — Error {resp.status_code}: {resp.text}")
                return []

        except requests.exceptions.RequestException as e:
            wait_time = BACKOFF_FACTOR * (2 ** retries)
            print(f"⚠️ {plot_no} — Network error: {e}. Retrying in {wait_time}s...")
            time.sleep(wait_time)
            retries += 1

    print(f"❌ {plot_no} — Max retries reached. Skipping.")
    return []


# ------------------------------------------
# EXECUTION BLOCK
# ------------------------------------------
print("🚀 Fetching weather data from Visual Crossing...")
records = []
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(fetch_weather, row) for _, row in df.iterrows()]
    for future in as_completed(futures):
        records.extend(future.result())

weather_df = pd.DataFrame(records)
print(f"🎯 Done! Retrieved {len(weather_df)} weather records across {weather_df['Plot_No'].nunique()} plots.")

weather_df.head()

🚀 Fetching weather data from Visual Crossing...
⚠️ 4 — Rate limit hit (429). Retrying in 5s...
⚠️ 5 — Rate limit hit (429). Retrying in 5s...
⚠️ 3 — Rate limit hit (429). Retrying in 5s...
⚠️ 1 — Rate limit hit (429). Retrying in 5s...
⚠️ 2 — Rate limit hit (429). Retrying in 5s...
⚠️ 39 — Rate limit hit (429). Retrying in 5s...
⚠️ 71 — Rate limit hit (429). Retrying in 5s...
⚠️ 66 — Rate limit hit (429). Retrying in 5s...
⚠️ 76 — Rate limit hit (429). Retrying in 5s...
⚠️ 105 — Rate limit hit (429). Retrying in 5s...
⚠️ 139 — Rate limit hit (429). Retrying in 5s...
⚠️ 140 — Rate limit hit (429). Retrying in 5s...
⚠️ 144 — Rate limit hit (429). Retrying in 5s...
⚠️ 195 — Rate limit hit (429). Retrying in 5s...
⚠️ 199 — Rate limit hit (429). Retrying in 5s...
⚠️ 200 — Rate limit hit (429). Retrying in 5s...
⚠️ 216 — Rate limit hit (429). Retrying in 5s...
⚠️ 249 — Rate limit hit (429). Retrying in 5s...
⚠️ 250 — Rate limit hit (429). Retrying in 5s...
⚠️ 274 — Rate limit hit (429). Retr

,Plot_No,Latitude,Longitude,Planting_Date,Date,Max_Temp_C,Min_Temp_C,Mean_Temp_C,Precip_mm,Humidity_%,SolarEnergy_MJm2,Evapotranspiration_mm,WindSpeed_kmh,CloudCover_%
0,6,-0.54792,37.484666,2021-10-21,2021-10-21,23.3,16.0,19.7,8.092,76.6,15.3,None,11.0,83.2
1,6,-0.54792,37.484666,2021-10-21,2021-10-22,26.6,16.6,21.2,1.500,70.4,18.3,None,9.3,65.4
2,6,-0.54792,37.484666,2021-10-21,2021-10-23,27.0,17.3,21.9,1.700,66.4,20.5,None,14.5,43.5
3,6,-0.54792,37.484666,2021-10-21,2021-10-24,28.6,14.6,22.3,0.000,61.3,24.7,None,14.6,20.0
4,6,-0.54792,37.484666,2021-10-21,2021-10-25,28.5,16.1,22.3,0.700,64.3,23.6,None,11.5,54.6


In [29]:
# 🧩 Number of unique plots after Visual Crossing extraction
unique_plots = weather_df['Plot_No'].nunique()
unique_plots

428

In [30]:
# Count missing values per column
missing_summary = weather_df.isna().sum().reset_index()
missing_summary.columns = ['Column', 'Missing_Count']
missing_summary['Missing_%'] = (missing_summary['Missing_Count'] / len(weather_df)) * 100

In [31]:
# Display nicely sorted output
missing_summary = missing_summary.sort_values(by='Missing_%', ascending=False)
missing_summary

,Column,Missing_Count,Missing_%
11,Evapotranspiration_mm,64481,100.0
0,Plot_No,0,0.0
1,Latitude,0,0.0
2,Longitude,0,0.0
3,Planting_Date,0,0.0
4,Date,0,0.0
5,Max_Temp_C,0,0.0
6,Min_Temp_C,0,0.0
7,Mean_Temp_C,0,0.0
8,Precip_mm,0,0.0


# **Determining Crop Stages**

**Define the GDD calculation function**

In [32]:

def daily_gdd(max_temp, min_temp, base_temp=10):
    # Convert temperatures to float in case they are strings
    try:
        max_temp = float(max_temp)
        min_temp = float(min_temp)
    except ValueError:
        raise ValueError(f"Invalid temperature value: max_temp={max_temp}, min_temp={min_temp}")
    adjusted_avg_temp = (max_temp + min_temp) / 2 
    
    # Calculate the Growing Degree Days (GDD) for a single day.
    return max(adjusted_avg_temp - base_temp, 0)

# **Compute the cumulative GDD for each day starting from a specified date**

In [33]:
def compute_cumulative_gdd(df):
    # Convert 'Date' and 'planting_date' columns to datetime format
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')
    df['Planting_Date'] = pd.to_datetime(df['Planting_Date'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')
    
    # Initialize an empty DataFrame to store results
    df_filtered_all = pd.DataFrame()
    
    # Group the DataFrame by Plot_No
    for plot_no, plot_data in df.groupby('Plot_No'):
        # Get the planting date for the current plot
        planting_date = plot_data['Planting_Date'].iloc[0]
        
        # Filter the DataFrame for dates from the planting date onwards
        df_filtered = plot_data[plot_data['Date'] >= planting_date].copy()
        
        # Calculate the daily GDD for each row and add it as a new column 'GDD'
        df_filtered['GDD'] = df_filtered.apply(lambda row: daily_gdd(row['Max_Temp_C'], row['Min_Temp_C']), axis=1)
        
        # Compute cumulative GDD for the current plot
        df_filtered['Cumulative_GDD'] = df_filtered['GDD'].cumsum()
        
        # Append the filtered and processed data to the result DataFrame
        df_filtered_all = pd.concat([df_filtered_all, df_filtered])

    # Save the DataFrame with cumulative GDD to a new CSV file
    output_csv_file = 'Cumulative_GDD3.csv'
    df_filtered_all.to_csv(output_csv_file, index=False)
   
    return df_filtered_all


In [34]:

df_GDD=compute_cumulative_gdd(weather_df)
df_GDD.head()

,Plot_No,Latitude,Longitude,Planting_Date,Date,Max_Temp_C,Min_Temp_C,Mean_Temp_C,Precip_mm,Humidity_%,SolarEnergy_MJm2,Evapotranspiration_mm,WindSpeed_kmh,CloudCover_%,GDD,Cumulative_GDD
2399,1,-0.499127,37.612253,2021-10-06,2021-10-06,26.9,12.0,20.9,1.5,61.9,24.8,None,15.7,48.4,9.45,9.45
2400,1,-0.499127,37.612253,2021-10-06,2021-10-07,25.5,15.7,20.2,5.2,67.4,23.2,None,16.2,70.1,10.60,20.05
2401,1,-0.499127,37.612253,2021-10-06,2021-10-08,23.3,16.4,20.0,14.0,74.5,20.5,None,16.6,68.4,9.85,29.90
2402,1,-0.499127,37.612253,2021-10-06,2021-10-09,24.3,16.6,20.6,1.2,70.8,12.4,None,11.2,82.6,10.45,40.35
2403,1,-0.499127,37.612253,2021-10-06,2021-10-10,26.4,17.5,21.3,0.7,71.6,16.7,None,17.9,81.9,11.95,52.30


In [35]:
output="weather_df.csv"
weather_df.to_csv(output,index=False)

In [36]:
weather_df.columns

Index(['Plot_No', 'Latitude', 'Longitude', 'Planting_Date', 'Date',
       'Max_Temp_C', 'Min_Temp_C', 'Mean_Temp_C', 'Precip_mm', 'Humidity_%',
       'SolarEnergy_MJm2', 'Evapotranspiration_mm', 'WindSpeed_kmh',
       'CloudCover_%'],
      dtype='object')

In [37]:
merged_df=pd.read_csv('merged_df.csv')
merged_df.head()

,Plot_No,Latitude,Longitude,Data_Planting_Date,Date,Max_Temp_C,Min_Temp_C,Mean_Temp_C,Precip_mm,Humidity_%,SolarEnergy_MJm2,Evapotranspiration_mm,WindSpeed_kmh,CloudCover_%,Planting_Date
0,3,-0.535121,37.60865,12/2/2021,12/2/2021,21.3,15.7,18.9,5.596,86.2,13.5,NaN,6.5,81.0,10/7/2021
1,3,-0.535121,37.60865,12/2/2021,12/3/2021,21.3,16.4,18.3,35.309,89.0,9.7,NaN,8.4,79.0,10/7/2021
2,3,-0.535121,37.60865,12/2/2021,12/4/2021,23.6,15.9,19.3,16.566,85.1,16.6,NaN,11.9,64.2,10/7/2021
3,3,-0.535121,37.60865,12/2/2021,12/5/2021,23.3,16.4,19.7,7.245,80.9,18.4,NaN,15.0,55.7,10/7/2021
4,3,-0.535121,37.60865,12/2/2021,12/6/2021,24.8,15.1,19.8,11.766,79.3,20.5,NaN,10.8,70.2,10/7/2021


In [38]:
import pandas as pd

def determine_corn_crop_stages_with_ideal_gdd(df):
    # Convert 'Date' and 'planting_date' to datetime format
    df['Date'] = pd.to_datetime(df['Date'])
    df['Planting_Date'] = pd.to_datetime(df['Planting_Date'])

    # Define crop stages with their GDD thresholds
    crop_stages = {
        "Emergence (VE)": 100,
        "V2 (Two leaves with collars visible)": 250,
        "V6 (Six leaves with collars visible)": 475,
        "VT (Tasseling)": 900,
        "R1 (Silking)": 1100,
        "R3 (Milk)": 1400,
        "R5 (Dent)": 1925,
        "R6 (Maturity)": 2450
    }

    # Initialize a list to store the rows for the dataframe
    crop_stage_data = []

    # Group DataFrame by Plot_No (assuming each plot has unique coordinates and planting date)
    for plot_no, plot_data in df.groupby('Plot_No'):
        # Get the planting date for the current plot
        planting_date = plot_data['Planting_Date'].iloc[0]
        Latitude=plot_data['Latitude'].iloc[1]
        Longitude=plot_data['Longitude'].iloc[2]

        # Compute cumulative GDD from the planting date
        df_gdd = compute_cumulative_gdd(plot_data)

        # For each crop stage, determine the date and GDD
        for stage, gdd_threshold in crop_stages.items():
            stage_date = df_gdd[df_gdd['Cumulative_GDD'] >= gdd_threshold].head(1)['Date']
            if not stage_date.empty:
                stage_date_value = stage_date.values[0]
                stage_gdd_value = df_gdd[df_gdd['Date'] == stage_date_value]['Cumulative_GDD'].values[0]
                
                # Append the data to the list
                crop_stage_data.append({
                    'plot_no': plot_no,
                    'Latitude':Latitude,
                    'Longitude':Longitude,
                    'planting_date':planting_date,
                    'Crop_Stage': stage,
                    'Stage_Date': stage_date_value,
                    'GDD': stage_gdd_value,
                    'Ideal_GDD': gdd_threshold  # Add the Ideal GDD value here
                })

    # Convert the list to a DataFrame
    crop_stage_df = pd.DataFrame(crop_stage_data)

    return crop_stage_df


In [39]:
vcross_df=determine_corn_crop_stages_with_ideal_gdd(merged_df)
vcross_df

,plot_no,Latitude,Longitude,planting_date,Crop_Stage,Stage_Date,GDD,Ideal_GDD
0,1,-0.499127,37.612253,2021-10-07,Emergence (VE),2021-10-16,112.30,100
1,1,-0.499127,37.612253,2021-10-07,V2 (Two leaves with collars visible),2021-10-29,259.90,250
2,1,-0.499127,37.612253,2021-10-07,V6 (Six leaves with collars visible),2021-11-17,477.50,475
3,1,-0.499127,37.612253,2021-10-07,VT (Tasseling),2021-12-29,903.40,900
4,1,-0.499127,37.612253,2021-10-07,R1 (Silking),2022-01-18,1101.65,1100
...,...,...,...,...,...,...,...,...
2364,428,-0.774969,37.066322,2021-10-22,Emergence (VE),2021-10-30,100.20,100
2365,428,-0.774969,37.066322,2021-10-22,V2 (Two leaves with collars visible),2021-11-13,254.70,250
2366,428,-0.774969,37.066322,2021-10-22,V6 (Six leaves with collars visible),2021-12-05,484.70,475
2367,428,-0.774969,37.066322,2021-10-22,VT (Tasseling),2022-01-17,901.45,900


In [40]:
stage_output="crop_stages_kenya.csv"
vcross_df.to_csv(stage_output,index=False)

In [41]:
import pandas as pd

# --- Assume you already have these two DataFrames ---
# weather_df  (with Data_Planting_Date and Planting_Date)
# stage_df    (with Crop_Stage and Stage_Date)

# 1️⃣ Normalize column names
merged_df.rename(columns={'Plot_No': 'plot_no'}, inplace=True)
vcross_df['Stage_Date'] = pd.to_datetime(vcross_df['Stage_Date'])
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Planting_Date'] = pd.to_datetime(merged_df['Planting_Date'])

# 2️⃣ Get latest stage per plot
latest_stage = vcross_df.sort_values('Stage_Date').groupby('plot_no').tail(1)
latest_stage = latest_stage[['plot_no', 'Crop_Stage', 'Stage_Date']]

# 3️⃣ Prepare an empty list to hold summary records
records = []

# 4️⃣ Loop through each plot and compute aggregated weather metrics
for _, row in latest_stage.iterrows():
    plot = row['plot_no']
    latest_stage_date = row['Stage_Date']
    crop_stage = row['Crop_Stage']

    # planting date for that plot
    planting_date = merged_df.loc[merged_df['plot_no'] == plot, 'Planting_Date'].iloc[0]

    # filter weather data within that range
    w = merged_df[(merged_df['plot_no'] == plot) &
                   (merged_df['Date'] >= planting_date) &
                   (merged_df['Date'] <= latest_stage_date)]

    if len(w) == 0:
        continue  # skip if no data

    # Aggregate
    record = {
        'plot_no': plot,
        'latitude': w['Latitude'].iloc[0],
        'longitude': w['Longitude'].iloc[0],
        'Data_Planting_Date': w['Data_Planting_Date'].iloc[0],
        'Planting_Date': planting_date,
        'Latest_Stage': crop_stage,
        'Latest_Stage_Date': latest_stage_date,
        'Max_Temp_C': w['Max_Temp_C'].max(),
        'Min_Temp_C': w['Min_Temp_C'].min(),
        'Mean_Temp_C': w['Mean_Temp_C'].mean(),
        'Precip_mm': w['Precip_mm'].sum(),
        'Humidity_%': w['Humidity_%'].mean(),
        'SolarEnergy_MJm2': w['SolarEnergy_MJm2'].sum(),
        'Evapotranspiration_mm': w['Evapotranspiration_mm'].sum(skipna=True),
        'WindSpeed_kmh': w['WindSpeed_kmh'].mean(),
        'CloudCover_%': w['CloudCover_%'].mean()
    }

    records.append(record)

# 5️⃣ Create final summary DataFrame
summary_df = pd.DataFrame(records)

# 6️⃣ Preview
summary_df.head()


,plot_no,latitude,longitude,Data_Planting_Date,Planting_Date,Latest_Stage,Latest_Stage_Date,Max_Temp_C,Min_Temp_C,Mean_Temp_C,Precip_mm,Humidity_%,SolarEnergy_MJm2,Evapotranspiration_mm,WindSpeed_kmh,CloudCover_%
0,145,0.118367,37.728812,10/7/2020,2020-09-05,R6 (Maturity),2021-05-22,30.8,8.9,20.950439,1329.017,72.812719,4971.9,0.0,12.819298,52.488158
1,139,0.219597,37.788483,10/10/2020,2020-09-06,R6 (Maturity),2021-05-23,30.7,8.9,20.867257,1323.000,72.303982,4966.7,0.0,13.486726,52.800885
2,230,0.302884,38.085174,10/20/2020,2020-09-04,R6 (Maturity),2021-05-23,32.2,8.9,21.790278,1319.300,74.532870,4726.7,0.0,13.418519,54.375000
3,411,-0.897704,37.026625,10/14/2020,2020-09-08,R6 (Maturity),2021-06-04,29.7,9.7,20.076068,829.114,71.237607,4887.9,0.0,24.261538,67.326923
4,112,-0.473318,37.415947,10/18/2020,2020-10-14,R6 (Maturity),2021-06-07,30.1,9.6,20.512446,1235.535,72.881116,4694.1,0.0,12.799142,60.377682


In [42]:
unique_plots = summary_df['plot_no'].nunique()
unique_plots

412

In [43]:
# --- Merge harvest data into summary_df ---
final_df = pd.merge(
    summary_df,
    df[['plot_no', 'crop_type', 'wet_harvest_kg/ha', 'dry_harvest_kg/ha']],
    on=["plot_no"],
    how='left'
)

# --- Preview final dataframe ---
final_df.head()

,plot_no,latitude,longitude,Data_Planting_Date,Planting_Date,Latest_Stage,Latest_Stage_Date,Max_Temp_C,Min_Temp_C,Mean_Temp_C,Precip_mm,Humidity_%,SolarEnergy_MJm2,Evapotranspiration_mm,WindSpeed_kmh,CloudCover_%,crop_type,wet_harvest_kg/ha,dry_harvest_kg/ha
0,145,0.118367,37.728812,10/7/2020,2020-09-05,R6 (Maturity),2021-05-22,30.8,8.9,20.950439,1329.017,72.812719,4971.9,0.0,12.819298,52.488158,maize,2853.75,2328.75
1,139,0.219597,37.788483,10/10/2020,2020-09-06,R6 (Maturity),2021-05-23,30.7,8.9,20.867257,1323.000,72.303982,4966.7,0.0,13.486726,52.800885,maize,526.88,198.75
2,230,0.302884,38.085174,10/20/2020,2020-09-04,R6 (Maturity),2021-05-23,32.2,8.9,21.790278,1319.300,74.532870,4726.7,0.0,13.418519,54.375000,maize,1402.50,1006.25
3,411,-0.897704,37.026625,10/14/2020,2020-09-08,R6 (Maturity),2021-06-04,29.7,9.7,20.076068,829.114,71.237607,4887.9,0.0,24.261538,67.326923,maize,5417.50,2305.62
4,112,-0.473318,37.415947,10/18/2020,2020-10-14,R6 (Maturity),2021-06-07,30.1,9.6,20.512446,1235.535,72.881116,4694.1,0.0,12.799142,60.377682,maize,4610.00,2598.12


In [44]:
# Drop the unwanted columns
final_for_yield = final_df.drop(columns=[
    "Data_Planting_Date",
    "Latest_Stage",
    "Latest_Stage_Date",
    "Evapotranspiration_mm"
])

In [45]:
final_for_yield.head()

,plot_no,latitude,longitude,Planting_Date,Max_Temp_C,Min_Temp_C,Mean_Temp_C,Precip_mm,Humidity_%,SolarEnergy_MJm2,WindSpeed_kmh,CloudCover_%,crop_type,wet_harvest_kg/ha,dry_harvest_kg/ha
0,145,0.118367,37.728812,2020-09-05,30.8,8.9,20.950439,1329.017,72.812719,4971.9,12.819298,52.488158,maize,2853.75,2328.75
1,139,0.219597,37.788483,2020-09-06,30.7,8.9,20.867257,1323.000,72.303982,4966.7,13.486726,52.800885,maize,526.88,198.75
2,230,0.302884,38.085174,2020-09-04,32.2,8.9,21.790278,1319.300,74.532870,4726.7,13.418519,54.375000,maize,1402.50,1006.25
3,411,-0.897704,37.026625,2020-09-08,29.7,9.7,20.076068,829.114,71.237607,4887.9,24.261538,67.326923,maize,5417.50,2305.62
4,112,-0.473318,37.415947,2020-10-14,30.1,9.6,20.512446,1235.535,72.881116,4694.1,12.799142,60.377682,maize,4610.00,2598.12


In [46]:
final_for_yield.to_csv('final_for_yield.csv')

**Import Indices**

In [47]:
index_df=pd.read_csv('all_indices.csv')
index_df.head()

,Plot_No,Date,Mean_NDVI,Mean_EVI,Mean_SAVI,Mean_NDRE,Mean_NDWI,Mean_NDMI,Max_NDVI,Max_EVI,...,Min_NDRE,Min_NDWI,Min_NDMI,Std_Dev_NDVI,Std_Dev_EVI,Std_Dev_SAVI,Std_Dev_NDRE,Std_Dev_NDWI,Std_Dev_NDMI,Cumulative_NDVI
0,3,10/07/2021,0.315456,0.169707,0.473113,0.193602,-0.495265,-0.074549,0.332959,0.175669,...,0.167763,-0.507765,-0.095315,0.026222,0.015488,0.039328,0.015306,0.014207,0.009093,0.3155
1,3,10/12/2021,0.315456,0.169707,0.473113,0.193602,-0.495265,-0.074549,0.332959,0.175669,...,0.167763,-0.507765,-0.095315,0.026222,0.015488,0.039328,0.015306,0.014207,0.009093,0.6309
2,3,10/17/2021,0.333320,0.171813,0.499899,0.202536,-0.511514,-0.082881,0.346773,0.174626,...,0.178060,-0.523987,-0.101538,0.024542,0.013931,0.036807,0.012043,0.013925,0.008150,0.9642
3,3,10/22/2021,0.340673,0.184697,0.510929,0.208338,-0.504088,-0.078778,0.354052,0.187796,...,0.186298,-0.516319,-0.096873,0.023275,0.013660,0.034907,0.012426,0.013476,0.009825,1.3049
4,3,10/27/2021,0.352110,0.188441,0.528080,0.215876,-0.507386,-0.077881,0.360551,0.188966,...,0.194217,-0.517190,-0.095863,0.019946,0.011861,0.029915,0.012517,0.011916,0.010360,1.6570


In [48]:
df = index_df.copy()
df['Date'] = pd.to_datetime(df['Date'])

In [49]:
df = df.sort_values(['Plot_No', 'Date'])
# Columns that start with "Mean_"
mean_cols = [col for col in df.columns if col.startswith("Mean_")]

In [50]:
agg_dict = {col: 'sum' for col in mean_cols}
agg_dict['Cumulative_NDVI'] = 'last'

In [51]:
# Group by Plot_No
final_df = df.groupby('Plot_No').agg(agg_dict).reset_index()

final_df.head()

,Plot_No,Mean_NDVI,Mean_EVI,Mean_SAVI,Mean_NDRE,Mean_NDWI,Mean_NDMI,Cumulative_NDVI
0,1,6.819444,3.719848,10.227529,4.463286,-7.211514,1.572886,6.8194
1,2,9.580029,5.905028,14.366404,6.331487,-8.904260,3.485063,9.5800
2,3,4.781868,2.845189,7.171665,2.973770,-6.208784,-0.301703,4.7819
3,4,8.503830,5.336094,12.753115,5.995714,-8.596676,0.985690,8.5038
4,5,9.004005,5.087394,13.502884,6.120798,-8.974788,2.486243,9.0040


In [52]:
agg_df = final_df.rename(columns={'Plot_No': 'plot_no'})

# **Merge the index and weathe df**

In [53]:
merged_df = final_for_yield.merge(agg_df, on='plot_no', how='left')
merged_df.head()

,plot_no,latitude,longitude,Planting_Date,Max_Temp_C,Min_Temp_C,Mean_Temp_C,Precip_mm,Humidity_%,SolarEnergy_MJm2,...,crop_type,wet_harvest_kg/ha,dry_harvest_kg/ha,Mean_NDVI,Mean_EVI,Mean_SAVI,Mean_NDRE,Mean_NDWI,Mean_NDMI,Cumulative_NDVI
0,145,0.118367,37.728812,2020-09-05,30.8,8.9,20.950439,1329.017,72.812719,4971.9,...,maize,2853.75,2328.75,8.050688,11.513297,12.073975,5.325104,-8.078459,2.951477,8.0507
1,139,0.219597,37.788483,2020-09-06,30.7,8.9,20.867257,1323.000,72.303982,4966.7,...,maize,526.88,198.75,6.343849,3.194971,9.513950,3.798797,-7.624553,2.439587,6.3438
2,230,0.302884,38.085174,2020-09-04,32.2,8.9,21.790278,1319.300,74.532870,4726.7,...,maize,1402.50,1006.25,11.314518,6.294860,16.968715,7.431315,-11.223136,3.638672,11.3145
3,411,-0.897704,37.026625,2020-09-08,29.7,9.7,20.076068,829.114,71.237607,4887.9,...,maize,5417.50,2305.62,11.743474,8.983573,17.612568,7.812345,-10.966121,5.535510,11.7435
4,112,-0.473318,37.415947,2020-10-14,30.1,9.6,20.512446,1235.535,72.881116,4694.1,...,maize,4610.00,2598.12,7.560659,8.125853,11.339539,5.216997,-6.901054,3.690113,7.5607


In [54]:
display(merged_df.columns)


Index(['plot_no', 'latitude', 'longitude', 'Planting_Date', 'Max_Temp_C',
       'Min_Temp_C', 'Mean_Temp_C', 'Precip_mm', 'Humidity_%',
       'SolarEnergy_MJm2', 'WindSpeed_kmh', 'CloudCover_%', 'crop_type',
       'wet_harvest_kg/ha', 'dry_harvest_kg/ha', 'Mean_NDVI', 'Mean_EVI',
       'Mean_SAVI', 'Mean_NDRE', 'Mean_NDWI', 'Mean_NDMI', 'Cumulative_NDVI'],
      dtype='object')

In [55]:
merged_df.to_csv('final_dataset.csv')